In [157]:
import cvxpy as cvx
import numpy as np
import pandas as pd

In [158]:
import  os

import numpy as np
import pandas as pd
import pickle
from features import feature_map


In [159]:
out_overall = pickle.load(open('../data/input/all_regions.pkl', 'r'))

region = "Austin"

df = out_overall[region]
df = df.rename(columns={'house_num_rooms': 'num_rooms',
                                  'num_occupants': 'total_occupants',
                                  'difference_ratio_min_max': 'ratio_difference_min_max'})
all_feature_homes = df[(df.full_agg_available == 1) & (df.md_available == 1)][all_cols].dropna().index
#drop_rows_having_no_data
o = {}
for h in df.index:
    o[h]=len(df.ix[h][feature_map['Monthly+Static']].dropna())
num_features_ser = pd.Series(o)
drop_rows = num_features_ser[num_features_ser==0].index

df = df.drop(drop_rows)


#df = df[(df.full_agg_available == 1) & (df.md_available == 1)]

In [160]:
df.describe()

,hvac_1,hvac_2,hvac_3,hvac_4,hvac_5,hvac_6,hvac_7,hvac_8,hvac_9,hvac_10,...,md_available,full_agg_available,ratio_min_max,ratio_difference_min_max,variance,skew,kurtosis,p_25,p_50,p_75
count,414.000000,410.000000,413.000000,414.000000,408.000000,395.000000,368.000000,363.000000,357.000000,351.000000,...,516.000000,516.000000,308.000000,308.000000,308.000000,308.000000,308.000000,308.000000,308.000000,308.000000
mean,51.913350,40.720842,47.952794,117.311154,203.102126,396.804457,620.253400,686.067597,481.118025,240.950593,...,0.277132,0.596899,0.321309,0.321309,176112.025710,0.676967,-0.449430,683.292817,865.839990,1235.429694
std,150.412654,108.261847,77.938878,110.375640,153.458581,262.526981,344.738007,366.714044,262.528830,155.361829,...,0.448016,0.490997,0.109082,0.109082,167965.719117,0.487553,1.061878,418.000180,493.873163,611.366181
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.001717,0.000883,0.003433,0.004250,0.000000,...,0.000000,0.000000,0.000774,0.000774,7920.383440,-2.620439,-1.946933,151.604664,195.780441,271.021820
25%,0.044650,0.071187,3.636833,41.229540,88.052885,189.932487,355.854012,411.792679,281.603027,116.655872,...,0.000000,0.000000,0.259692,0.259692,70078.603306,0.488628,-1.083481,438.269007,553.534782,837.741047
50%,0.984917,1.797167,22.470232,91.265301,176.931725,342.428436,605.059448,670.011841,472.222900,220.287064,...,0.000000,1.000000,0.312210,0.312210,125329.434826,0.746568,-0.726823,607.740189,759.551514,1095.130753
75%,25.600200,26.878354,60.826897,160.078270,285.000435,570.531860,836.296936,907.554810,642.858459,349.301041,...,1.000000,1.000000,0.391264,0.391264,212519.280173,0.949711,-0.095873,807.661926,1041.041794,1512.081039
max,1032.923218,768.023865,715.312134,934.861694,1258.950684,1609.004517,2130.617920,1834.047852,1386.018921,876.380493,...,1.000000,1.000000,0.602784,0.602784,1294741.012680,2.037984,8.027035,3648.723206,4120.627197,4806.598511


In [161]:
from create_df_larger import read_df_larger
#df, dfc, all_homes, appliance_min, national_average = read_df_larger()

df = df.rename(columns={'house_num_rooms':'num_rooms',
                        'num_occupants':'total_occupants',
                        'difference_ratio_min_max':'ratio_difference_min_max'})

dfc = df.copy()
K_min, K_max = 1,6
F_min, F_max=1,8

from all_functions import *
from features_larger import *

In [162]:
appliance="fridge"
if appliance=="hvac":
    start, end=5, 11
else:
    start, end = 1, 13

appliance_cols = ['%s_%d' %(appliance, month) for month in range(start, end)]
aggregate_cols = ['aggregate_%d' %month for month in range(start, end)]

all_cols = deepcopy(appliance_cols)
all_cols.extend(aggregate_cols)
X_matrix = dfc[all_cols].dropna()
    

In [163]:
columns_max = {}
columns_min = {}
col_max = X_matrix.max()
columns_max[appliance]=col_max
col_min = X_matrix.min()
columns_min[appliance]=col_min
X_normalised = X_matrix.copy()
#for col in X_matrix.columns:
#    X_normalised[col] = (X_matrix[col]-col_min[col])/(col_max[col]-col_min[col])
for col in X_matrix.columns:
    X_normalised[col] = (X_matrix[col]-col_min.min())/(col_max.max()-col_min.min())



In [164]:
pd.DataFrame(X_normalised).head()

,fridge_1,fridge_2,fridge_3,fridge_4,fridge_5,fridge_6,fridge_7,fridge_8,fridge_9,fridge_10,...,aggregate_3,aggregate_4,aggregate_5,aggregate_6,aggregate_7,aggregate_8,aggregate_9,aggregate_10,aggregate_11,aggregate_12
26,0.010565,0.009035,0.008345,0.009840,0.009860,0.009414,0.010152,0.010349,0.010022,0.009993,...,0.114865,0.151019,0.185897,0.267958,0.330721,0.366107,0.264749,0.232218,0.140129,0.126031
59,0.001164,0.001052,0.001205,0.001221,0.001319,0.001293,0.001260,0.001234,0.001226,0.001319,...,0.298040,0.200372,0.214823,0.202411,0.273815,0.351388,0.199952,0.180123,0.206024,0.341312
86,0.009127,0.008833,0.009622,0.010927,0.011540,0.011258,0.013909,0.014344,0.013659,0.013853,...,0.185308,0.289290,0.237053,0.262816,0.335077,0.369049,0.293869,0.227132,0.163372,0.165459
93,0.008084,0.007539,0.008444,0.009257,0.009562,0.009410,0.010065,0.009947,0.009572,0.009549,...,0.082984,0.103717,0.140486,0.204904,0.277587,0.304961,0.231908,0.148019,0.090346,0.136379
94,0.009575,0.008578,0.010052,0.010005,0.010909,0.011210,0.011945,0.012446,0.011545,0.011517,...,0.057917,0.048296,0.059979,0.071015,0.099687,0.112883,0.088662,0.066861,0.070536,0.068103


In [165]:
df = pd.DataFrame(X_normalised)

In [166]:
df.head()

,fridge_1,fridge_2,fridge_3,fridge_4,fridge_5,fridge_6,fridge_7,fridge_8,fridge_9,fridge_10,...,aggregate_3,aggregate_4,aggregate_5,aggregate_6,aggregate_7,aggregate_8,aggregate_9,aggregate_10,aggregate_11,aggregate_12
26,0.010565,0.009035,0.008345,0.009840,0.009860,0.009414,0.010152,0.010349,0.010022,0.009993,...,0.114865,0.151019,0.185897,0.267958,0.330721,0.366107,0.264749,0.232218,0.140129,0.126031
59,0.001164,0.001052,0.001205,0.001221,0.001319,0.001293,0.001260,0.001234,0.001226,0.001319,...,0.298040,0.200372,0.214823,0.202411,0.273815,0.351388,0.199952,0.180123,0.206024,0.341312
86,0.009127,0.008833,0.009622,0.010927,0.011540,0.011258,0.013909,0.014344,0.013659,0.013853,...,0.185308,0.289290,0.237053,0.262816,0.335077,0.369049,0.293869,0.227132,0.163372,0.165459
93,0.008084,0.007539,0.008444,0.009257,0.009562,0.009410,0.010065,0.009947,0.009572,0.009549,...,0.082984,0.103717,0.140486,0.204904,0.277587,0.304961,0.231908,0.148019,0.090346,0.136379
94,0.009575,0.008578,0.010052,0.010005,0.010909,0.011210,0.011945,0.012446,0.011545,0.011517,...,0.057917,0.048296,0.059979,0.071015,0.099687,0.112883,0.088662,0.066861,0.070536,0.068103


In [167]:
def transform_2(pred_df,appliance,  col_max, col_min):
    pred_df_copy = pred_df.copy()
    if appliance=="hvac":
        start, stop=5, 11
    else:
        start, stop=1, 13
    
    for month in range(start, stop):
        pred_df_copy['%s_%d' %(appliance,month)] = (col_max.max()-col_min.min())*pred_df['%s_%d'%(appliance,month)] +col_min.min()
    return pred_df_copy

def transform(pred_df,appliance,  col_max, col_min):
    pred_df_copy = pred_df.copy()
    if appliance=="hvac":
        start, stop=5, 11
    else:
        start, stop=1, 13
    
    for month in range(start, stop):
        pred_df_copy['%s_%d' %(appliance,month)] = (col_max['%s_%d' %(appliance, month)]-col_min['%s_%d' %(appliance,month)])*pred_df['%s_%d'%(appliance,month)] +col_min['%s_%d' %(appliance,month)]
    return pred_df_copy



In [168]:
def nmf(A, k, MAX_ITERS=30):
    np.random.seed(0)

    # Generate random data matrix A.
    m = len(A)
    n = len(A.columns)
    
    
    # Initialize Y randomly.
    Y_init = np.random.rand(m, k)

    # Ensure same initial random Y, rather than generate new one
    # when executing this cell.
    Y = Y_init 

    # Perform alternating minimization.
    
    residual = np.zeros(MAX_ITERS)
    for iter_num in range(1, 1+MAX_ITERS):
        # At the beginning of an iteration, X and Y are NumPy
        # array types, NOT CVXPY variables.

        # For odd iterations, treat Y constant, optimize over X.
        if iter_num % 2 == 1:
            X = cvx.Variable(k, n)
            constraint = [X >= 0]
        # For even iterations, treat X constant, optimize over Y.
        else:
            Y = cvx.Variable(m, k)
            constraint = [Y >= 0]
        Temp = Y*X

        # Solve the problem.
        obj = cvx.Minimize(cvx.norm(A.values[mask] - (Y*X)[mask], 'fro'))
        prob = cvx.Problem(obj, constraint)
        prob.solve(solver=cvx.SCS)

        if prob.status != cvx.OPTIMAL:
            pass
            #return X.value, Y.value, residual
            #raise Exception("Solver did not converge!")

        #print 'Iteration {}, residual norm {}'.format(iter_num, prob.value)
        residual[iter_num-1] = prob.value

        # Convert variable to NumPy array constant for next iteration.
        if iter_num % 2 == 1:
            X = X.value
        else:
            Y = Y.value
    return X, Y, residual

In [169]:
out = {}
for lat in range(1,7):
    out[lat]={}
    print lat
    for home in X_normalised.index[:]:
        
        X_home = X_normalised.copy()
        for month in range(start, end):
            X_home.loc[home, '%s_%d' %(appliance, month)] = np.NAN
        mask = X_home.notnull().values
        # Ensure repeatably random problem data.
        A = X_home.copy()
        X, Y, res = nmf(A, lat, 10)
        
        pred_df = pd.DataFrame(Y*X)
        pred_df.columns = X_normalised.columns
        pred_df.index = X_normalised.index
        #out[lat][home] = pred_df.ix[home][appliance_cols]
        out[lat][home] = transform_2(pred_df.ix[home], appliance, col_max, col_min)[appliance_cols]


1
2
3
4
5
6


In [173]:
error_lat = {}
gt = X_matrix[appliance_cols]
for lat in range(1,7):
    
    pr = pd.DataFrame(out[lat]).T
    
    error = (pr-gt).abs()
    error_df = error.div(gt).mul(100)
    # Ignoring homes which have all features
    
    error_df_reduced = error_df.ix[[x for x in error_df.index if x not in all_feature_homes]]
    error_lat[lat] = error_df_reduced.dropna().median().mean()

In [174]:
error_lat

{1: 31.67251674811818,
 2: 32.53443040712421,
 3: 30.406127364478575,
 4: 29.718905143423594,
 5: 31.70345194451471,
 6: 31.178667244375617}

In [66]:
error.div(gt).mul(100).median().mean()

30.31132296948245

In [67]:
error_df.dropna().median().mean()

30.31132296948245

In [70]:
error_df_reduced

,fridge_1,fridge_2,fridge_3,fridge_4,fridge_5,fridge_6,fridge_7,fridge_8,fridge_9,fridge_10,fridge_11,fridge_12
86,18.077656,18.250714,14.706067,17.057206,14.541159,18.296025,29.516496,31.074985,30.376546,30.530239,31.251740,31.444905
93,17.153120,16.369231,19.481405,21.795547,26.587934,28.365342,25.546471,28.007269,27.434552,27.672743,21.001232,16.962338
114,33.081774,34.497924,27.994729,30.308907,31.952681,26.996189,25.840010,25.761254,24.303114,28.045382,27.601298,30.938289
115,14.922894,32.247620,8.495782,18.541511,12.554606,11.278482,11.196404,14.697628,16.129398,11.465693,9.787112,7.982988
160,39.444883,36.063531,31.609717,35.549668,39.766927,41.869373,35.783818,36.085133,40.189805,37.953146,34.365016,35.624610
187,68.031794,69.967537,62.903464,79.409273,75.893285,69.695770,75.791127,60.856734,76.501272,67.786881,64.277175,74.481007
434,6.799923,8.130511,1.165522,2.837884,35.694310,70.873513,90.369009,90.857400,93.148300,89.156940,64.156466,61.961069
457,70.040580,70.614925,70.980360,70.710857,70.026087,70.783173,68.812423,70.647945,69.746843,70.539096,70.321262,1970.918141
484,45.731438,33.793812,34.572337,26.570571,19.489803,20.938084,22.502760,27.892883,32.937991,30.880774,31.722219,39.274853
499,1229.596156,66.811359,69.097714,67.664968,64.927735,64.676829,65.473813,67.806446,64.781450,63.682494,65.949981,70.944158


In [49]:
df.columns

Index([u'fridge_1', u'fridge_2', u'fridge_3', u'fridge_4', u'fridge_5',
       u'fridge_6', u'fridge_7', u'fridge_8', u'fridge_9', u'fridge_10',
       u'fridge_11', u'fridge_12', u'aggregate_1', u'aggregate_2',
       u'aggregate_3', u'aggregate_4', u'aggregate_5', u'aggregate_6',
       u'aggregate_7', u'aggregate_8', u'aggregate_9', u'aggregate_10',
       u'aggregate_11', u'aggregate_12'],
      dtype='object')